# Case - RCA 集中性分析

Copyright © 2019 Hsu Shih-Chieh


In [6]:
%load_ext autoreload
%autoreload 2

from datasets import load_rca
import numpy as np
import pandas as pd
from functools import reduce
from IPython.display import display


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [27]:
data = load_rca() 
print(data.DESCR)

RCA dataset
---------------------------

**Data Set Characteristics:**

    :Number of Instances: 
        - test: 3468
        - sfc: 3308
        - parts: 16544
        
    :Number of Attributes: 
        - test: 13
        - sfc: 11
        - parts: 11
    
    :Test Attributes information:
        - SN: UUT序號
        - Station: 測試工站
        - Stationcode: 測試工站ID
        - Machine: 測試設備ID
        - start_time: 測試開始時間
        - end_time: 測試結束時間
        - isTestFail: 是否測試Fail
        - symptom: 測試徵狀
        - desc: 測試結果說明
        - uploadtime: 紀錄上傳時間
        - emp: 空欄位
        - ver1: 測試軟體版本號
        - ver2: 測試軟體版本號
        
    :SFC Attributes information: 
        - ID: SFC ID
        - SN: 產品序號
        - WO: 工單
        - HH_Part: 鴻海料號
        - CUST_Part: 客戶料號
        - assembline: 組裝線體
        - scantime: 掃描barcode時間
        - na1: 空欄位
        - na2: 空欄位 
        - product: 產品名稱
        - floor: 組裝樓層
        
    :PARTS Attributes information:         
        - ID: SFC ID
      

### 測試工站良率計算
- TLEOL的測試良率最差, 因為只有一個測試工站 

In [7]:
test_df = data.test
for p in  test_df['Station'].unique():
    failcnt  = test_df.groupby('Station').get_group(p).groupby('SN')['isTestFail'].sum()
    failsnlist = failcnt[np.where(failcnt>1)[0]].index.unique()
    test_df['isTrueFail'] = test_df.apply(lambda row: row['Station']==p and row['SN'] in list(failsnlist),axis=1)
    
station_gby = test_df.groupby(['Station'])
station_failgby = test_df[test_df['isTrueFail']==1].groupby(['Station'])
station = station_gby.groups.keys()
production = station_gby['SN'].nunique()
failqty = station_failgby['SN'].nunique()
yieldtable = pd.concat([failqty,production], axis=1)
yieldtable.columns=['fail qty','prouction']
yieldtable['failrate']=yieldtable['fail qty']/yieldtable['prouction']
yieldtable.fillna(0)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,fail qty,prouction,failrate
CSA_PackNShip,0.0,131,0.00000
TLEOL,23.0,3300,0.00697


**算Top1 Fail Station的Symptom Fail數量**
- 鎖定 Symptom: Scan3StreakFlare

In [14]:
symptom_gby = station_gby.get_group('TLEOL').fillna('NAN').drop_duplicates(subset=['SN','isTrueFail'])
symptom_gby = symptom_gby.where(tmp['isTrueFail']==1).groupby('symptom').sum()
symptom_gby
#有些事NAN所以symptom fail總數不等於23

,isTestFail,isTrueFail
symptom,,
NAN,4.0,4.0
Scan3GetTargetInfo.2; Scan3StreakFlare,1.0,1.0
Scan3StreakFlare,17.0,17.0
Scan3StreakRGB,1.0,1.0


**組大表**

In [20]:
partdict = {}
dfs=[]
sfc_df = data.sfc
parts_df = data.parts
#---- Test_station machine ----        
for p in  test_df['Station'].unique():
    d = test_df.groupby('Station').get_group(p)
    d=d[['SN','Machine']]
    d.columns=['ID','{}_Machine'.format(p)]
    partdict[p]=d
    dfs.append(d)
    
#---- SFC_WO ----            
sfc_gby = sfc_df.groupby('ID')    
wo_df = pd.DataFrame(sfc_gby['WO'].max()).reset_index()
dfs.append(wo_df)

#---- PART_PARTSN ----
for p in  parts_df['part'].unique():
    d = parts_df.groupby('part').get_group(p)
    d=d[['ID','PARTSN']]
    d.columns=['ID','{}_PARTSN'.format(p)]
    partdict[p]=d
    dfs.append(d)

#---- PART_OPID ----    
for p in  parts_df['part'].unique():
    d = parts_df.groupby('part').get_group(p)
    d=d[['ID','opid']]
    d.columns=['ID','{}_OPID'.format(p)]
    partdict[p]=d
    dfs.append(d)
    
df_final = reduce(lambda left,right: pd.merge(left,right,on='ID', how='outer'), dfs)
df_final = df_final.drop_duplicates('ID')
print(df_final.shape)
df_final.head()


(3309, 14)


,ID,CSA_PackNShip_Machine,TLEOL_Machine,WO,CD_PARTSN,C44_PARTSN,4BH6_PARTSN,CAL_PARTSN,LK_PARTSN,CD_OPID,C44_OPID,4BH6_OPID,CAL_OPID,LK_OPID
0,CN8BS7C3GT,LC_CSA_PackNShip_01,NaN,296205081.0,CD184600760,C448462PF3,4BH6618J13151094,CAL84648TD,C18X040A3AFAC,F1034891,F1034891,F1034891,F1034891,F1034891
1,CN8BS7C4GB,LC_CSA_PackNShip_01,NaN,296205081.0,CD18460076K,C448473B0Y,4BH6618J03032107,CAL84646FS,C18X020WUVFAC,F1034891,F1034891,F1034891,F1034891,F1034891
2,CN8BT7C02X,LC_CSA_PackNShip_01,NaN,296205546.0,CD18460076G,C44847376Q,4BH6618J04218019,CAL8464D49,C18X020NC2FAC,F1034891,F1034891,F1034891,F1034891,F1034891
3,CN8BT7C050,LC_CSA_PackNShip_01,LC_TLEOL_39,296205546.0,CD18460075K,C448473P61,4BH6618I10328068,CAL8470LDV,C18X020F6EFAC,F1034891,F1034891,F1034891,F1034891,F1034891
4,CN8BT7C060,LC_CSA_PackNShip_01,LC_TLEOL_29,296205546.0,CD18460075K,C448473NP2,4BH6618J03101061,CAL8470K7P,C18X0412YYFAC,F1034891,F1034891,F1034891,F1034891,F1034891


**根據物料序號編碼規則解析VENDOR與DATECODE**

In [21]:
df_final['LK_VENDOR'] = df_final['LK_PARTSN'].map(lambda sn:str(sn)[8:10])
df_final['CD_VENDOR'] = df_final['CD_PARTSN'].map(lambda sn:sn[0:2])
df_final['CD_DATECODE'] = df_final['CD_PARTSN'].map(lambda sn:'_'.join([sn[0:2], sn[2:6]]))   
df_final['CAL_VENDOR'] = df_final['CAL_PARTSN'].map(lambda sn:sn[0:3])
df_final['CAL_DATECODE'] = df_final['CAL_PARTSN'].map(lambda sn:'_'.join([sn[0:3], sn[3:6]]))
df_final['LK_VENDOR'] = df_final['LK_PARTSN'].map(lambda sn:str(sn)[8:10])
df_final['LK_DATECODE'] = df_final['LK_PARTSN'].map(lambda sn:'_'.join([str(sn)[8:10], str(sn)[1:5]]))
df_final['C44_VENDOR'] = df_final['C44_PARTSN'].map(lambda sn:sn[0:3])
df_final['C44_DATECODE'] = df_final['C44_PARTSN'].map(lambda sn:'_'.join([sn[0:3], sn[3:6]]))
df_final['4BH6_VENDOR'] = df_final['4BH6_PARTSN'].map(lambda sn:sn[0:4])
df_final['4BH6_DATECODE'] = df_final['4BH6_PARTSN'].map(lambda sn:'_'.join([sn[0:4], sn[9:14]]))



**標記是否為Symptom Fail**
- symptom: Scan3StreakFlare

In [23]:
failsnlist = test_df[test_df['isTrueFail']==1].drop_duplicates(['SN','Station']).groupby('symptom').get_group('Scan3StreakFlare')['SN'].unique()
df_final['isSymptomFail'] = df_final['ID'].map(lambda sn: 1 if sn in failsnlist else 0)
df_final = df_final.fillna('NA')


**集中性運算**

In [31]:
riskfactor = df_final.columns
riskfactor = list(filter(lambda x: ('OPID' in x) or ('Machine' in x) or ('VENDOR' in x) or ('DATECODE' in x), riskfactor))
for r in riskfactor:
    #Fail Qty
    fqty = df_final.groupby(r)['isSymptomFail'].sum()
    total_fqty = pd.Series(data=[fqty.sum()]*len(fqty.index), index=fqty.index)
    #Production Qty
    pqty = df_final.groupby(r)['isSymptomFail'].count()
    total_pqty = pd.Series(data=[pqty.sum()]*len(pqty.index), index=pqty.index)
    rcatable = pd.concat([fqty,total_fqty,pqty,total_pqty], axis=1)
    rcatable.columns=['fail qty','total fail qty','prouction qty','total production qty']
    rcatable['commonality']= rcatable['fail qty']/rcatable['total fail qty']
    rcatable['produciton ratio']= rcatable['prouction qty']/rcatable['total production qty']
    display(rcatable)

,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
CSA_PackNShip_Machine,,,,,,
LC_CSA_PackNShip_01,0,17,131,3309,0.0,0.039589
NA,17,17,3178,3309,1.0,0.960411


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
TLEOL_Machine,,,,,,
LC_TLEOL_01,0,17,74,3309,0.000000,0.022363
LC_TLEOL_02,0,17,71,3309,0.000000,0.021457
LC_TLEOL_03,0,17,69,3309,0.000000,0.020852
LC_TLEOL_04,1,17,62,3309,0.058824,0.018737
LC_TLEOL_05,0,17,64,3309,0.000000,0.019341
LC_TLEOL_06,0,17,72,3309,0.000000,0.021759
LC_TLEOL_07,0,17,75,3309,0.000000,0.022665
LC_TLEOL_08,0,17,73,3309,0.000000,0.022061
LC_TLEOL_09,0,17,75,3309,0.000000,0.022665


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
CD_OPID,,,,,,
F1034891,17,17,3309,3309,1.0,1.0


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
C44_OPID,,,,,,
F1034891,17,17,3309,3309,1.0,1.0


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
4BH6_OPID,,,,,,
F1034891,17,17,3309,3309,1.0,1.0


...

,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
LK_DATECODE,,,,,,
00_18X0,0,17,5,3309,0.000000,0.001511
01_18X0,0,17,5,3309,0.000000,0.001511
02_18X0,0,17,6,3309,0.000000,0.001813
03_18X0,0,17,6,3309,0.000000,0.001813
04_18X0,0,17,3,3309,0.000000,0.000907
05_18X0,0,17,3,3309,0.000000,0.000907
06_18X0,0,17,7,3309,0.000000,0.002115
07_18X0,0,17,5,3309,0.000000,0.001511
08_18X0,0,17,4,3309,0.000000,0.001209


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
C44_VENDOR,,,,,,
C44,17,17,3309,3309,1.0,1.0


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
C44_DATECODE,,,,,,
C44_838,0,17,1,3309,0.000000,0.000302
C44_845,14,17,21,3309,0.823529,0.006346
C44_846,3,17,160,3309,0.176471,0.048353
C44_847,0,17,3127,3309,0.000000,0.944998


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
4BH6_VENDOR,,,,,,
4BH6,15,17,3304,3309,0.882353,0.998489
CAL8,2,17,4,3309,0.117647,0.001209
FJ88,0,17,1,3309,0.000000,0.000302


,fail qty,total fail qty,prouction qty,total production qty,commonality,produciton ratio
4BH6_DATECODE,,,,,,
4BH6_00630,0,17,1,3309,0.000000,0.000302
4BH6_01400,0,17,2,3309,0.000000,0.000604
4BH6_01620,0,17,16,3309,0.000000,0.004835
4BH6_01621,0,17,4,3309,0.000000,0.001209
4BH6_01670,0,17,44,3309,0.000000,0.013297
4BH6_01671,0,17,25,3309,0.000000,0.007555
4BH6_01791,0,17,1,3309,0.000000,0.000302
4BH6_01800,0,17,16,3309,0.000000,0.004835
4BH6_01801,0,17,2,3309,0.000000,0.000604
